## Can hexapod motions during the exposure explain the guider drifts?

Craig Lage  02-Dec-25

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from lsst.daf.butler import Butler
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.simonyi.mountData import MountData
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils import (
    ConsDbClient,
    getAirmassSeeingCorrection,
    getBandpassSeeingCorrection,
)


In [ ]:
def HexapodDrifts(mountData: MountData) -> tuple[float, float]:
    """Calculate the image drift associated with the hexapod motions
    for a given exposure.
    Parameters
    ----------
    mountData : MountData
        The EFD data associated with the exposure
    Returns
    -------
    tuple[float, float]
        The image motions associated with the CamHex and M2Hex motions.
        (AltDrift, AzDrift)
    """
    # The below image motion coefficients were calculated
    # with a Batoid simulation by Josh Meyers
    camHexXY = 1.00  # microns(image) / micron(hexapod)
    camHexUV = 4.92  # microns(image) / arcsecond(hexapod)
    m2HexXY = 1.13  # microns(image) / micron(hexapod)
    m2HexUV = 37.26  # microns(image) / arcsecond(hexapod)

    # Convert these to image impact in arcseconds
    # The 10.0 is microns / pixel
    pixelScale = 0.2  # arcseconds / pixel - find this elsewhere?
    camHexXY = camHexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
    camHexUV = camHexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
    # Convention here is [Alt, Az] at rotator=0
    camCoefs = np.array([[camHexXY,0.0], [0.0, -camHexXY], \
                        [0.0,0.0], [0.0, camHexUV], [camHexUV, 0.0]])
    m2HexXY = m2HexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
    m2HexUV = m2HexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
    # Convention here is [Alt, Az] at rotator=0
    m2Coefs = np.array([[m2HexXY,0.0], [0.0, -m2HexXY], \
                        [0.0,0.0], [m2HexUV, 0.0], [0.0, m2HexUV]])

    drift = (0.0, 0.0)

    # For each hexapod motion, fit a line and determine the slope
    camhex = mountData.camhexData
    times = camhex['private_kafkaStamp']
    times -= times.iloc[0]
    for i in [0, 1, 3, 4]: # Z has no impact
        dat = camhex[f'position{i}']
        fit = np.polyfit(times, dat, 1)
        change = fit[0] * times.iloc[-1]  * camCoefs[i]
        #print(f"Cam, {i}, {fit[0]}, {change}")
        drift += change

    m2hex = mountData.m2hexData
    times = m2hex['private_kafkaStamp']
    times -= times.iloc[0]
    for i in [0, 1, 3, 4]: # Z has no impact
        dat = m2hex[f'position{i}']
        fit = np.polyfit(times, dat, 1)
        change = fit[0] * times.iloc[-1] * m2Coefs[i]
        #print(f"M2, {i}, {fit[0]}, {change}")
        drift += change

    # Now rotate it into the AltAz coords.
    rot = np.median(mountData.rotationData['actualPosition'].values)
    theta = np.pi * rot / 180.0
    R = [[ np.cos(theta), -np.sin(theta)],
     [ np.sin(theta),  np.cos(theta)]]
    drift = tuple(np.dot(R, drift)) #(Alt, Az)
    return drift

In [ ]:
client = makeEfdClient()
butler = Butler('/repo/embargo', collections=["LSSTCam/raw/all", "LSSTCam/calib", \
                                        "LSSTCam/runs/nightlyValidation"])
instrument = 'LSSTCam'

In [ ]:
def fetch(client, day_obs, seq_min, seq_max):
    
    visits_query = f'''
    SELECT 
    ccdvisit1_quicklook.psf_sigma,
    ccdvisit1_quicklook.psf_ixx,
    ccdvisit1_quicklook.psf_iyy,
    ccdvisit1_quicklook.psf_ixy,
    ccdvisit1.detector,
    visit1_quicklook.aos_fwhm,
    visit1.visit_id,
    visit1.seq_num,
    visit1.day_obs
    FROM
    cdb_lsstcam.ccdvisit1_quicklook AS ccdvisit1_quicklook,
    cdb_lsstcam.ccdvisit1 AS ccdvisit1,
    cdb_lsstcam.visit1_quicklook AS visit1_quicklook,
    cdb_lsstcam.visit1 AS visit1 
    WHERE 
    ccdvisit1.ccdvisit_id = ccdvisit1_quicklook.ccdvisit_id
    AND ccdvisit1.visit_id = visit1.visit_id 
    AND visit1.visit_id = visit1_quicklook.visit_id
    AND ccdvisit1.detector NOT IN (168, 188, 123, 27, 0, 20, 65, 161)
    AND visit1.day_obs = {day_obs}
    AND (visit1.seq_num BETWEEN {seq_min} AND {seq_max})
    '''
    
    ccdvisits = client.query(visits_query).to_pandas()

    pixel_scale = 0.2
    sig2fwhm = 2 * np.sqrt(2 * np.log(2))
    ccdvisits["psf_fwhm"] = ccdvisits["psf_sigma"] * sig2fwhm * pixel_scale
    ccdvisits["psf_fwhm"] = pd.to_numeric(ccdvisits["psf_fwhm"], errors="coerce")
    ccdvisits["psf_ixx"] = pd.to_numeric(ccdvisits["psf_ixx"], errors="coerce")
    ccdvisits["psf_iyy"] = pd.to_numeric(ccdvisits["psf_iyy"], errors="coerce")
    ccdvisits["psf_ixy"] = pd.to_numeric(ccdvisits["psf_ixy"], errors="coerce")
    ccdvisits["aos_fwhm"] = pd.to_numeric(ccdvisits["aos_fwhm"], errors="coerce")
    denom = ccdvisits["psf_ixx"] + ccdvisits["psf_iyy"]
    denom = denom.replace(0, np.nan)
    
    e1 = (ccdvisits["psf_ixx"] - ccdvisits["psf_iyy"]) / denom
    e2 = (2 * ccdvisits["psf_ixy"]) / denom

    ccdvisits["ellipticity"] = np.sqrt(e1.to_numpy()**2 + e2.to_numpy()**2)

    return ccdvisits

In [ ]:
os.environ["no_proxy"] += ",.consdb"
cdb_client = ConsDbClient('http://consdb-pq.consdb:8080/consdb')

#dayObs = 20251209
#table = fetch(cdb_client, dayObs, 286, 392)
#dayObs = 20251210
#table = fetch(cdb_client, dayObs, 63, 83)
dayObs = 20251211
table = fetch(cdb_client, dayObs, 108, 128)
filtered_table = table[table['day_obs'] == dayObs]
raw_filtered_table = table[table['day_obs'] == dayObs]
filtered_table = filtered_table.select_dtypes(include="number")
filtered_table = filtered_table.groupby("seq_num").agg({col: 'first' if col == 'band' else 'mean' for col in filtered_table.columns})
hex_alt_drifts = []
hex_az_drifts = []
guider_alt_drifts = []
guider_az_drifts = []
psfs = []
ells = []
aoss = []

expIds = []
seqNums = []
# json data copied from the summit as it's not in consDB yet
guiderTable = pd.read_json(f'/home/c/cslage/u/LSSTCam/rubintv_data/dayObs_{dayObs}.json').T
guiderTable = guiderTable.sort_index()
print(dayObs, len(guiderTable))
for i in range(108, 128):#range(1, len(guiderTable)+1):
    if (i > 305) and (i < 373):
        continue
    #try:
    expId = int(dayObs * 1.0E5 + i)
    dataId = {'exposure':expId, 'instrument':'LSSTCam'}
    expRecord = getExpRecordFromDataId(butler, dataId)
    (mountErrors, mountData) = calculateMountErrors(expRecord, client)
    (hex_alt_drift, hex_az_drift) = HexapodDrifts(mountData)
    guider_alt_drift = float(guiderTable.loc[i]['Alt drift (arcsec total)']) * 30.0
    guider_az_drift = float(guiderTable.loc[i]['Az drift (arcsec total)']) * 30.0
    hex_alt_drifts.append(hex_alt_drift)
    hex_az_drifts.append(hex_az_drift)
    guider_alt_drifts.append(guider_alt_drift)
    guider_az_drifts.append(guider_az_drift)
    seqNums.append(i)
    expIds.append(expId)
    ell = filtered_table[filtered_table['seq_num'] == i]['ellipticity'].values[0]
    ells.append(ell)
    psf = filtered_table[filtered_table['seq_num'] == i]['psf_fwhm'].values[0]
    psfs.append(psf)
    aos = filtered_table[filtered_table['seq_num'] == i]['aos_fwhm'].values[0]
    aoss.append(aos)
    print(f"{expId} succeeded!")
    #except:
    #    print(f"{expId} failed!")
    #    continue
    


In [ ]:
xaxis = np.arange(0, len(seqNums), 1)
#xticks = [0, 9, 19, 29, 39]
#xticklabels = [286, 296, 373, 383, 392]
xticks = [0, 9]
xticklabels = [108, 118]
plt.figure(figsize=(20, 10))
plt.subplots_adjust(hspace=0.3)
plt.suptitle(f"Guider drifts vs Hexapod drifts {dayObs}", fontsize=18)
plt.subplot(2,4,1)
plt.title("Azimuth drift/exposure")
plt.scatter(xaxis, guider_az_drifts, marker='x',s=10, color='blue', label="Guider_drifts")
plt.plot(xaxis, hex_az_drifts, marker='o', ms=1, color='red', alpha=0.5, lw=4, label="Hexapod induced drift")
plt.axvline(9.5, ls='--', color='black')
#plt.axvline(19.5, ls='--', color='black')
#plt.axvline(29.5, ls='--', color='black')
plt.xticks(ticks=xticks, labels=xticklabels)
plt.xlabel("Sequence number")
plt.ylabel("Drift/exposure (arcsec)")
plt.ylim(-0.6, 0.6)
plt.legend(loc = 'lower left')
plt.subplot(2,4,2)
plt.title("Elevation drift/exposure")
plt.scatter(xaxis, guider_alt_drifts, marker='x', s=10, color='blue', label="Guider_drifts")
plt.plot(xaxis, hex_alt_drifts, marker='o', ms=1, color='red', alpha=0.5, lw=4, label="Hexapod induced drift")
plt.axvline(9.5, ls='--', color='black')
#plt.axvline(19.5, ls='--', color='black')
#plt.axvline(29.5, ls='--', color='black')
plt.xticks(ticks=xticks, labels=xticklabels)
plt.xlabel("Sequence number")
plt.ylabel("Drift/exposure (arcsec)")
plt.ylim(-0.6, 0.6)
plt.legend(loc = 'upper right')
xs = np.linspace(-0.6, 0.6, 100)
plt.subplot(2,4,5)
with_hex_motion = []
no_hex_motion = []
for drift, seq in zip(guider_az_drifts, seqNums):
    #if seq < 296 or ((seq > 305) and (seq < 383)):
    if seq < 118:
        with_hex_motion.append(drift)
    else:
        no_hex_motion.append(drift)
plt.title("Azimuth drift/exposure")
plt.hist(with_hex_motion, bins=8, color='blue', label='With hex motion')
plt.hist(no_hex_motion, bins=8, color='red', label='No hex motion')
plt.ylabel("Guider drift (arcsec/exposure)")
plt.xlim(-0.6, 0.6)
plt.legend(loc='upper left')
plt.subplot(2,4,6)
with_hex_motion = []
no_hex_motion = []
for drift, seq in zip(guider_alt_drifts, seqNums):
    #if seq < 296 or ((seq > 305) and (seq < 383)):
    if seq < 118:
        with_hex_motion.append(drift)
    else:
        no_hex_motion.append(drift)

plt.title("Elevation drift/exposure")
plt.hist(with_hex_motion, bins=8, color='blue', label='With hex motion')
plt.hist(no_hex_motion, bins=8, color='red', label='No hex motion')
plt.ylabel("Guider drift (arcsec/exposure)")
plt.xlim(-0.6, 0.6)
plt.legend(loc='upper left')

plt.subplot(2,4,3)
plt.title("PSF FWHM")
plt.scatter(xaxis, psfs, marker='x', s=10, color='blue')
plt.axvline(9.5, ls='--', color='black')
#plt.axvline(19.5, ls='--', color='black')
#plt.axvline(29.5, ls='--', color='black')
plt.xticks(ticks=xticks, labels=xticklabels)
plt.xlabel("Sequence number")
plt.ylabel("PSF FWHM (arcsec)")
plt.text(1, 0.9, "Move")
plt.text(11, 0.9, "No Move")
#plt.text(21, 0.8, "Move")
#plt.text(31, 0.8, "No Move")
plt.subplot(2,4,4)
plt.title("Ellipticity")
plt.scatter(xaxis, ells, marker='x', s=10, color='blue')
plt.axvline(9.5, ls='--', color='black')
#plt.axvline(19.5, ls='--', color='black')
#plt.axvline(29.5, ls='--', color='black')
plt.xticks(ticks=xticks, labels=xticklabels)
plt.xlabel("Sequence number")
plt.ylabel("Ellipticity")
plt.text(1, 0.04, "Move")
plt.text(11, 0.04, "No Move")
#plt.text(21, 0.1, "Move")
#plt.text(31, 0.1, "No Move")
plt.subplot(2,4,7)
plt.title("AOS FWHM")
plt.scatter(xaxis, aoss, marker='x', s=10, color='blue')
plt.axvline(9.5, ls='--', color='black')
#plt.axvline(19.5, ls='--', color='black')
#plt.axvline(29.5, ls='--', color='black')
plt.xticks(ticks=xticks, labels=xticklabels)
plt.xlabel("Sequence number")
plt.ylabel("AOS FWHM")
plt.text(1, 0.40, "Move")
plt.text(11, 0.40, "No Move")
#plt.text(21, 0.46, "Move")
#plt.text(31, 0.46, "No Move")

plt.savefig(f"/home/c/cslage/u/LSSTCam/guider_data/Guider_Drifts_vs_Hexapod_3_{dayObs}.png")

In [ ]:
with_hex_motion = []
no_hex_motion = []
for drift, seq in zip(guider_az_drifts, seqNums):
    if seq < 296:
        with_hex_motion.append(drift)
    else:
        no_hex_motion.append(drift)

In [ ]:
startDay = 20251122
endDay = 20251201
pdf = PdfPages(f"/home/c/cslage/u/LSSTCam/guider_data/Guider_Drifts_vs_Hexapod_{startDay}_{endDay}.pdf")
fig = plt.figure(figsize=(10, 10))
dayObs = startDay
while dayObs <= endDay:
    hex_alt_drifts = []
    hex_az_drifts = []
    guider_alt_drifts = []
    guider_az_drifts = []
    expIds = []
    seqNums = []
    guiderTable = pd.read_json(f'/home/c/cslage/u/LSSTCam/rubintv_data/dayObs_{dayObs}.json').T
    guiderTable = guiderTable.sort_index()
    print(dayObs, len(guiderTable))
    for i in range(1, len(guiderTable)+1):
        try:
            expId = int(dayObs * 1.0E5 + i)
            dataId = {'exposure':expId, 'instrument':'LSSTCam'}
            expRecord = getExpRecordFromDataId(butler, dataId)
            (mountErrors, mountData) = calculateMountErrors(expRecord, client)
            (hex_alt_drift, hex_az_drift) = HexapodDrifts(mountData)
            guider_alt_drift = float(guiderTable.loc[i]['Alt drift (arcsec total)']) * 30.0
            guider_az_drift = float(guiderTable.loc[i]['Az drift (arcsec total)']) * 30.0
            hex_alt_drifts.append(hex_alt_drift)
            hex_az_drifts.append(hex_az_drift)
            guider_alt_drifts.append(guider_alt_drift)
            guider_az_drifts.append(guider_az_drift)
            seqNums.append(i)
            expIds.append(expId)
            print(f"{expId} succeeded!")
        except:
            print(f"{expId} failed!")
            continue

    plt.subplots_adjust(hspace=0.3)
    plt.suptitle(f"Guider drifts vs Hexapod drifts {dayObs}", fontsize=18)
    plt.subplot(2,2,1)
    plt.title("Azimuth drift/exposure")
    plt.plot(seqNums, guider_az_drifts, marker='x', ms=1, color='blue', alpha=0.5, label="Guider_drifts")
    plt.plot(seqNums, hex_az_drifts, marker='o', ms=1, color='red', alpha=0.5, lw=4, label="Hexapod induced drift")
    plt.xlabel("Sequence number")
    plt.ylabel("Drift/exposure (arcsec)")
    plt.ylim(-0.6, 0.6)
    plt.legend(loc = 'lower left')
    plt.subplot(2,2,2)
    plt.title("Elevation drift/exposure")
    plt.plot(seqNums, guider_alt_drifts, marker='x', ms=1, color='blue', alpha=0.5, label="Guider_drifts")
    plt.plot(seqNums, hex_alt_drifts, marker='o', ms=1, color='red', alpha=0.5, lw=4, label="Hexapod induced drift")
    plt.xlabel("Sequence number")
    plt.ylabel("Drift/exposure (arcsec)")
    plt.ylim(-0.6, 0.6)
    plt.legend(loc = 'lower left')
    xs = np.linspace(-0.6, 0.6, 100)
    plt.subplot(2,2,3)
    plt.title("Azimuth drift/exposure")
    plt.scatter(hex_az_drifts, guider_az_drifts, marker='x', s=10, color='blue')
    plt.plot(xs, xs, ls='--', color='red')
    plt.xlabel("Hexapod induced drift (arcsec/exposure)")
    plt.ylabel("Guider drift (arcsec/exposure)")
    plt.xlim(-0.6, 0.6)
    plt.ylim(-0.6, 0.6)
    plt.subplot(2,2,4)
    plt.title("Elevation drift/exposure")
    plt.scatter(hex_alt_drifts, guider_alt_drifts, marker='x', s=10, color='blue')
    plt.plot(xs, xs, ls='--', color='red')
    plt.xlabel("Hexapod induced drift (arcsec/exposure)")
    plt.ylabel("Guider drift (arcsec/exposure)")
    plt.xlim(-0.6, 0.6)
    plt.ylim(-0.6, 0.6)
    pdf.savefig(fig)
    fig.clf()
    print(f"Finished {dayObs}")
    dayObs = calcNextDay(dayObs)
pdf.close()

In [ ]:
guiderTable.columns

In [ ]:
expRecord.